In [64]:
import cv2
import os
import numpy as np
from skimage.feature import graycomatrix, graycoprops
from skimage.filters import gabor
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [65]:
   # Function for lesion segmentation
def segment_lesion(image):
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply adaptive thresholding to segment the lesion
    _, segmented_lesion = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # Perform morphological operations to clean up the segmented region
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    segmented_lesion = cv2.morphologyEx(segmented_lesion, cv2.MORPH_CLOSE, kernel)
    
    # Bitwise AND with the original image to obtain the segmented lesion
    segmented_lesion = cv2.bitwise_and(image, image, mask=segmented_lesion)
    
    return segmented_lesion



In [66]:

# Function to extract features from segmented lesions
def extract_features(lesion_image):
    # Calculate GLCM features
    gray_image = cv2.cvtColor(lesion_image, cv2.COLOR_BGR2GRAY)
    glcm = graycomatrix(gray_image, distances=[5], angles=[0], symmetric=True, normed=True)
    contrast = graycoprops(glcm, prop='contrast')[0, 0]
    correlation = graycoprops(glcm, prop='correlation')[0, 0]
    energy = graycoprops(glcm, prop='energy')[0, 0]
    homogeneity = graycoprops(glcm, prop='homogeneity')[0, 0]
    
    # Calculate Gabor features
    frequency = 0.6
    theta = 1.5
    gabor_features, _ = gabor(gray_image, frequency=frequency, theta=theta)
    mean_squared_energy = np.mean(gabor_features ** 2)
    mean_amplitude = np.mean(np.abs(gabor_features))
    
    # Combine all features into a feature vector
    feature_vector = [contrast, correlation, energy, homogeneity,mean_squared_energy, mean_amplitude]
    return feature_vector

In [107]:
def load_and_augment_images(directory):
    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest'
    )

    preprocessed_images = []
    labels = []
    
    for filename in os.listdir(directory):
        image_path = os.path.join(directory, filename)
        img = cv2.imread(image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB
        
        img = np.expand_dims(img, axis=0)
        aug_iter = datagen.flow(img)
        
        # Generate augmented images
        num_augmented_images = 10  # Number of augmented images per original image
        for _ in range(num_augmented_images):
            augmented_image = aug_iter.next()[0].astype(np.uint8)
            segmented_lesion = segment_lesion(augmented_image)
            feature_vector = extract_features(segmented_lesion)
            preprocessed_images.append(feature_vector)
            labels.append(1 if 'oral_cancer' in filename else 0)
    
    print(f"Number of images loaded: {len(preprocessed_images)}")
    return np.array(preprocessed_images), np.array(labels)

images_directory = 'E:\ocd\Images'
preprocessed_images, labels = load_and_augment_images(images_directory)

Number of images loaded: 2520


In [108]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(preprocessed_images, labels, test_size=0.25, random_state=42)
len(X_train)

1890

In [109]:
# Naive Bayes
nb_classifier = GaussianNB()
nb_classifier.fit(X_train, y_train)
nb_accuracy = nb_classifier.score(X_test, y_test)
print("Naive Bayes Accuracy:", nb_accuracy)

Naive Bayes Accuracy: 0.6206349206349207


In [110]:
# K-Nearest Neighbors (KNN)
knn_classifier = KNeighborsClassifier(n_neighbors=3)
knn_classifier.fit(X_train, y_train)
knn_accuracy = knn_classifier.score(X_test, y_test)
print("KNN Accuracy:", knn_accuracy)

KNN Accuracy: 0.5761904761904761


In [111]:
# Support Vector Machine (SVM)
svm_classifier = SVC(kernel='linear')
svm_classifier.fit(X_train, y_train)
svm_accuracy = svm_classifier.score(X_test, y_test)
print("SVM Accuracy:", svm_accuracy)

SVM Accuracy: 0.6158730158730159
